# 0. General

In [1]:
# Import libraries to be used

# Warning messages display
## import warnings
## warnings.filterwarnings(action='ignore') # https://docs.python.org/3/library/warnings.html#the-warnings-filter

# Directories/Files management
import os.path
## from zipfile import ZipFile # De momento no ha hecho falta 

# Data analysis and wrangling
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # Show all columns in DataFrames
## pd.set_option('display.max_rows', None) # It greatly slows down the output display and freezes the kernel

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot') # choose a style: 'plt.style.available'
sns.set_theme(context='notebook',
              style="darkgrid") # {darkgrid, whitegrid, dark, white, ticks}
palette = sns.color_palette("flare", as_cmap=True);
## import altair as alt

# Machine Learning
## from sklearn.[...] import ...

In [2]:
# Detect Operating System running and manage paths accordingly

if os.name == 'nt': # Windows
    root = r"C:\Users\turge\CompartidoVM\0.TFM"
    print("Running on Windows.")
elif os.name == 'posix': # Ubuntu
    root = "/home/dsc/shared/0.TFM"
    print("Running on Ubuntu.")
print("root path\t", root)

Running on Windows.
root path	 C:\Users\turge\CompartidoVM\0.TFM


# 1. A/L On-Time Performance

In [3]:
int_cols = [
#             'YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE', # Time Period
            'OP_UNIQUE_CARRIER', 'OP_CARRIER_AIRLINE_ID', 'OP_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM', # Airline, A/C and Flight
            'ORIGIN_CITY_MARKET_ID', 'ORIGIN', 'ORIGIN_CITY_NAME', # Origin
            'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME', # Destination
            'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', # Departure Performance
            'WHEELS_ON', 'TAXI_IN', 'ARR_TIME', 'ARR_DELAY', # Arrival Performance
            'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', # Cancellations and Diversions
            'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'FLIGHTS', 'DISTANCE', 'DISTANCE_GROUP', # Flight Summaries
            'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', # Cause of Delay
#             'DIV_AIRPORT_LANDINGS', 'DIV_REACHED_DEST', 'DIV_ACTUAL_ELAPSED_TIME', 'DIV_ARR_DELAY', 'DIV_DISTANCE'] # Diverted Airport Information
            ]

41 columns are still quite a few, but more than half of them (68) have been dropped.

Additional information on each column meaning can be found [here](https://www.transtats.bts.gov/Fields.asp?Table_ID=236&SYS_Table_Name=T_ONTIME_REPORTING&User_Table_Name=Reporting%20Carrier%20On-Time%20Performance%20(1987-present)&Year_Info=1&First_Year=1987&Last_Year=2020&Rate_Info=0&Frequency=Monthly&Data_Frequency=Annual,Quarterly,Monthly).

Once the "interesting" columns are selected, the files shall be imported again.

Let's proceed with multiple-file importing, through concatenation into a single DataFrame.

In [4]:
directory_in_str = os.path.join(root,
                                "Raw_Data",
                                "US_DoT",
                                "ONTIME_REPORTING")
directory_in_str

'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Raw_Data\\US_DoT\\ONTIME_REPORTING'

In [5]:
# List the files' paths corresponding to each month of year 2019

file_list = []
try:
    os.listdir(directory_in_str)
except FileNotFoundError:
    print("The system cannot find the specified path:\n" + directory_in_str + "\nPlease check the path has been properly set.")
else:
    for file in os.listdir(directory_in_str):
        if file.endswith(".zip") and file.startswith("19"): # Year 2019
            file_list.append(os.path.join(directory_in_str, file))
            continue
        else:
            continue
file_list    

['C:\\Users\\turge\\CompartidoVM\\0.TFM\\Raw_Data\\US_DoT\\ONTIME_REPORTING\\1901_921771952_T_ONTIME_REPORTING.zip',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Raw_Data\\US_DoT\\ONTIME_REPORTING\\1902_921771952_T_ONTIME_REPORTING.zip',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Raw_Data\\US_DoT\\ONTIME_REPORTING\\1903_921771952_T_ONTIME_REPORTING.zip',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Raw_Data\\US_DoT\\ONTIME_REPORTING\\1904_921771952_T_ONTIME_REPORTING.zip',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Raw_Data\\US_DoT\\ONTIME_REPORTING\\1905_921881115_T_ONTIME_REPORTING.zip',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Raw_Data\\US_DoT\\ONTIME_REPORTING\\1906_921881115_T_ONTIME_REPORTING.zip',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Raw_Data\\US_DoT\\ONTIME_REPORTING\\1907_921881115_T_ONTIME_REPORTING.zip',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Raw_Data\\US_DoT\\ONTIME_REPORTING\\1908_921888367_T_ONTIME_REPORTING.zip',
 'C:\\Users\\turge\\CompartidoVM\\0.TFM\\Raw_Data\\US_DoT\\ONTIM

In [6]:
%%time

# Create a DataFrame from the 12 month-files corresponding to the year 2019

df7 = pd.DataFrame()
for i, csv_path in enumerate(file_list):
    if i == 13: # Fail-safe: in case the list captured more than 12 files
        break
    df_month = pd.read_csv(csv_path,
                           encoding='latin1',
                           nrows=1e6, # Fail-safe: in case the file is inadvertently too big
                           usecols=int_cols, # This way, the extra column is disregarded for the loading process
                           low_memory = False) # This will prevent from auto-dtypes
    df7 = df7.append(df_month)
df7

Wall time: 1min 55s


,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,9E,20363,9E,N195PQ,5121,35412,TYS,"Knoxville, TN",30397,ATL,"Atlanta, GA",1205.0,25.0,30.0,1235.0,1311.0,4.0,1315.0,25.0,0.0,NaN,0.0,70.0,70.0,36.0,1.0,152.0,1,0.0,0.0,0.0,0.0,25.0
1,9E,20363,9E,N919XJ,5121,35412,TYS,"Knoxville, TN",30397,ATL,"Atlanta, GA",1250.0,70.0,35.0,1325.0,1403.0,9.0,1412.0,82.0,0.0,NaN,0.0,70.0,82.0,38.0,1.0,152.0,1,0.0,0.0,12.0,0.0,70.0
2,9E,20363,9E,N316PQ,5122,30397,ATL,"Atlanta, GA",34783,SGF,"Springfield, MO",956.0,6.0,20.0,1016.0,1040.0,3.0,1043.0,-8.0,0.0,NaN,0.0,121.0,107.0,84.0,1.0,563.0,3,NaN,NaN,NaN,NaN,NaN
3,9E,20363,9E,N325PQ,5122,30397,ATL,"Atlanta, GA",34783,SGF,"Springfield, MO",945.0,-5.0,16.0,1001.0,1026.0,3.0,1029.0,-24.0,0.0,NaN,0.0,123.0,104.0,85.0,1.0,563.0,3,NaN,NaN,NaN,NaN,NaN
4,9E,20363,9E,N904XJ,5122,30397,ATL,"Atlanta, GA",34783,SGF,"Springfield, MO",947.0,-3.0,25.0,1012.0,1040.0,4.0,1044.0,-9.0,0.0,NaN,0.0,123.0,117.0,88.0,1.0,563.0,3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625758,B6,20409,B6,N193JB,846,31454,MCO,"Orlando, FL",31703,SWF,"Newburgh/Poughkeepsie, NY",1500.0,64.0,20.0,1520.0,1726.0,5.0,1731.0,52.0,0.0,NaN,0.0,163.0,151.0,126.0,1.0,989.0,4,52.0,0.0,0.0,0.0,0.0
625759,B6,20409,B6,N304JB,854,30852,DCA,"Washington, DC",30721,BOS,"Boston, MA",1414.0,-6.0,15.0,1429.0,1526.0,7.0,1533.0,-17.0,0.0,NaN,0.0,90.0,79.0,57.0,1.0,399.0,2,NaN,NaN,NaN,NaN,NaN
625760,B6,20409,B6,N193JB,860,34100,PHL,"Philadelphia, PA",30721,BOS,"Boston, MA",652.0,-8.0,12.0,704.0,746.0,5.0,751.0,-34.0,0.0,NaN,0.0,85.0,59.0,42.0,1.0,280.0,2,NaN,NaN,NaN,NaN,NaN
625761,B6,20409,B6,N563JB,861,30721,BOS,"Boston, MA",34819,SJU,"San Juan, PR",812.0,-1.0,10.0,822.0,1245.0,3.0,1248.0,-27.0,0.0,NaN,0.0,242.0,216.0,203.0,1.0,1674.0,7,NaN,NaN,NaN,NaN,NaN


In [7]:
for i,col in enumerate(df7.columns):
    print(i, col)

0 OP_UNIQUE_CARRIER
1 OP_CARRIER_AIRLINE_ID
2 OP_CARRIER
3 TAIL_NUM
4 OP_CARRIER_FL_NUM
5 ORIGIN_CITY_MARKET_ID
6 ORIGIN
7 ORIGIN_CITY_NAME
8 DEST_CITY_MARKET_ID
9 DEST
10 DEST_CITY_NAME
11 DEP_TIME
12 DEP_DELAY
13 TAXI_OUT
14 WHEELS_OFF
15 WHEELS_ON
16 TAXI_IN
17 ARR_TIME
18 ARR_DELAY
19 CANCELLED
20 CANCELLATION_CODE
21 DIVERTED
22 CRS_ELAPSED_TIME
23 ACTUAL_ELAPSED_TIME
24 AIR_TIME
25 FLIGHTS
26 DISTANCE
27 DISTANCE_GROUP
28 CARRIER_DELAY
29 WEATHER_DELAY
30 NAS_DELAY
31 SECURITY_DELAY
32 LATE_AIRCRAFT_DELAY


In [8]:
df7.insert(loc=24,
           column='ACTUAL-CRS_ELAPSED_TIME',
           value= df7['ACTUAL_ELAPSED_TIME'] - df7['CRS_ELAPSED_TIME'])
df7

,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,ACTUAL-CRS_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,9E,20363,9E,N195PQ,5121,35412,TYS,"Knoxville, TN",30397,ATL,"Atlanta, GA",1205.0,25.0,30.0,1235.0,1311.0,4.0,1315.0,25.0,0.0,NaN,0.0,70.0,70.0,0.0,36.0,1.0,152.0,1,0.0,0.0,0.0,0.0,25.0
1,9E,20363,9E,N919XJ,5121,35412,TYS,"Knoxville, TN",30397,ATL,"Atlanta, GA",1250.0,70.0,35.0,1325.0,1403.0,9.0,1412.0,82.0,0.0,NaN,0.0,70.0,82.0,12.0,38.0,1.0,152.0,1,0.0,0.0,12.0,0.0,70.0
2,9E,20363,9E,N316PQ,5122,30397,ATL,"Atlanta, GA",34783,SGF,"Springfield, MO",956.0,6.0,20.0,1016.0,1040.0,3.0,1043.0,-8.0,0.0,NaN,0.0,121.0,107.0,-14.0,84.0,1.0,563.0,3,NaN,NaN,NaN,NaN,NaN
3,9E,20363,9E,N325PQ,5122,30397,ATL,"Atlanta, GA",34783,SGF,"Springfield, MO",945.0,-5.0,16.0,1001.0,1026.0,3.0,1029.0,-24.0,0.0,NaN,0.0,123.0,104.0,-19.0,85.0,1.0,563.0,3,NaN,NaN,NaN,NaN,NaN
4,9E,20363,9E,N904XJ,5122,30397,ATL,"Atlanta, GA",34783,SGF,"Springfield, MO",947.0,-3.0,25.0,1012.0,1040.0,4.0,1044.0,-9.0,0.0,NaN,0.0,123.0,117.0,-6.0,88.0,1.0,563.0,3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625758,B6,20409,B6,N193JB,846,31454,MCO,"Orlando, FL",31703,SWF,"Newburgh/Poughkeepsie, NY",1500.0,64.0,20.0,1520.0,1726.0,5.0,1731.0,52.0,0.0,NaN,0.0,163.0,151.0,-12.0,126.0,1.0,989.0,4,52.0,0.0,0.0,0.0,0.0
625759,B6,20409,B6,N304JB,854,30852,DCA,"Washington, DC",30721,BOS,"Boston, MA",1414.0,-6.0,15.0,1429.0,1526.0,7.0,1533.0,-17.0,0.0,NaN,0.0,90.0,79.0,-11.0,57.0,1.0,399.0,2,NaN,NaN,NaN,NaN,NaN
625760,B6,20409,B6,N193JB,860,34100,PHL,"Philadelphia, PA",30721,BOS,"Boston, MA",652.0,-8.0,12.0,704.0,746.0,5.0,751.0,-34.0,0.0,NaN,0.0,85.0,59.0,-26.0,42.0,1.0,280.0,2,NaN,NaN,NaN,NaN,NaN
625761,B6,20409,B6,N563JB,861,30721,BOS,"Boston, MA",34819,SJU,"San Juan, PR",812.0,-1.0,10.0,822.0,1245.0,3.0,1248.0,-27.0,0.0,NaN,0.0,242.0,216.0,-26.0,203.0,1.0,1674.0,7,NaN,NaN,NaN,NaN,NaN


In [9]:
df7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7422037 entries, 0 to 625762
Data columns (total 34 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   OP_UNIQUE_CARRIER        object 
 1   OP_CARRIER_AIRLINE_ID    int64  
 2   OP_CARRIER               object 
 3   TAIL_NUM                 object 
 4   OP_CARRIER_FL_NUM        int64  
 5   ORIGIN_CITY_MARKET_ID    int64  
 6   ORIGIN                   object 
 7   ORIGIN_CITY_NAME         object 
 8   DEST_CITY_MARKET_ID      int64  
 9   DEST                     object 
 10  DEST_CITY_NAME           object 
 11  DEP_TIME                 float64
 12  DEP_DELAY                float64
 13  TAXI_OUT                 float64
 14  WHEELS_OFF               float64
 15  WHEELS_ON                float64
 16  TAXI_IN                  float64
 17  ARR_TIME                 float64
 18  ARR_DELAY                float64
 19  CANCELLED                float64
 20  CANCELLATION_CODE        object 
 21  DIVERTED 

In [10]:
'ORIGIN_CITY_MARKET_ID', 'ORIGIN', 'ORIGIN_CITY_NAME', # Origin
'DEST_CITY_MARKET_ID', 'DEST', 'DEST_CITY_NAME', # Destination
'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', # Departure Performance
'WHEELS_ON', 'TAXI_IN', 'ARR_TIME', 'ARR_DELAY', # Arrival Performance
'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', # Cancellations and Diversions
'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'FLIGHTS', 'DISTANCE', 'DISTANCE_GROUP', # Flight Summaries
'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', # Cause of Delay

('CARRIER_DELAY',
 'WEATHER_DELAY',
 'NAS_DELAY',
 'SECURITY_DELAY',
 'LATE_AIRCRAFT_DELAY')

In [49]:
%%time

mode = lambda x: x.value_counts().index[0]

OTP_carrier_2019 = df7.groupby(['OP_CARRIER_AIRLINE_ID', 'OP_UNIQUE_CARRIER', 'OP_CARRIER']) \
                      .agg({'ORIGIN_CITY_MARKET_ID' : ['count'],
                            'ORIGIN' : ['count'],
                            'DEST_CITY_MARKET_ID' : ['count'],
                            'DEST' : ['count'],
                            'DEP_DELAY' : ['sum', 'mean', 'min', 'max', 'median'],
                            'TAXI_OUT' : ['sum', 'mean', 'median'],
                            'TAXI_IN' : ['sum', 'mean', 'median'],
                            'ARR_DELAY' : ['sum', 'mean', 'min', 'max', 'median'],
                            'CANCELLED' : ['sum', ('mode', mode)],
                            'DIVERTED' : ['sum', ('mode', mode)],
                            'ACTUAL-CRS_ELAPSED_TIME' : ['sum', 'mean', 'min', 'max', 'median'],
                            'AIR_TIME' : ['sum', 'mean', 'median'],
                            'FLIGHTS' : [('mode', mode)],
                            'DISTANCE' : ['sum', 'mean', 'median']
                           })
OTP_carrier_2019

Wall time: 6.03 s


ORIGIN_CITY_MARKET_ID  \
                                                                   count   
OP_CARRIER_AIRLINE_ID OP_UNIQUE_CARRIER OP_CARRIER                         
19393                 WN                WN                       1363946   
19690                 HA                HA                         83891   
19790                 DL                DL                        991986   
19805                 AA                AA                        946776   
19930                 AS                AS                        264816   
19977                 UA                UA                        625910   
20304                 OO                OO                        836445   
20363                 9E                9E                        257132   
20366                 EV                EV                        134683   
20368                 G4                G4                        105305   
20378                 YV                YV                        227888   
20397                 OH                OH                        289304   
20398                 MQ                MQ                        327007   
20409                 B6                B6                        297411   
20416                 NK                NK                        204845   
20436                 F9                F9                        135543   
20452                 YX                YX                        329149   

                                                     ORIGIN  \
                                                      count   
OP_CARRIER_AIRLINE_ID OP_UNIQUE_CARRIER OP_CARRIER            
19393                 WN                WN          1363946   
19690                 HA                HA            83891   
19790                 DL                DL           991986   
19805                 AA                AA           946776   
19930                 AS                AS           264816   
19977                 UA                UA           625910   
20304                 OO                OO           836445   
20363                 9E                9E           257132   
20366                 EV                EV           134683   
20368                 G4                G4           105305   
20378                 YV                YV           227888   
20397                 OH                OH           289304   
20398                 MQ                MQ           327007   
20409                 B6                B6           297411   
20416                 NK                NK           204845   
20436                 F9                F9           135543   
20452                 YX                YX           329149   

                                                   DEST_CITY_MARKET_ID  \
                                                                 count   
OP_CARRIER_AIRLINE_ID OP_UNIQUE_CARRIER OP_CARRIER                       
19393                 WN                WN                     1363946   
19690                 HA                HA                       83891   
19790                 DL                DL                      991986   
19805                 AA                AA                      946776   
19930                 AS                AS                      264816   
19977                 UA                UA                      625910   
20304                 OO                OO                      836445   
20363                 9E                9E                      257132   
20366                 EV                EV                      134683   
20368                 G4                G4                      105305   
20378                 YV                YV                      227888   
20397                 OH                OH                      289304   
20398                 MQ                MQ                      327007   
20409                 B6                B6                      297411   
20416      

In [38]:
OTP_carrier_2019.reset_index(inplace=True)
OTP_carrier_2019

OP_CARRIER_AIRLINE_ID OP_UNIQUE_CARRIER OP_CARRIER ORIGIN_CITY_MARKET_ID  \
                                                                      count   
0                  19393                WN         WN               1363946   
1                  19690                HA         HA                 83891   
2                  19790                DL         DL                991986   
3                  19805                AA         AA                946776   
4                  19930                AS         AS                264816   
5                  19977                UA         UA                625910   
6                  20304                OO         OO                836445   
7                  20363                9E         9E                257132   
8                  20366                EV         EV                134683   
9                  20368                G4         G4                105305   
10                 20378                YV         YV                227888   
11                 20397                OH         OH                289304   
12                 20398                MQ         MQ                327007   
13                 20409                B6         B6                297411   
14                 20416                NK         NK                204845   
15                 20436                F9         F9                135543   
16                 20452                YX         YX                329149   

     ORIGIN DEST_CITY_MARKET_ID     DEST   DEP_DELAY                           \
      count               count    count         sum       mean   min     max   
0   1363946             1363946  1363946  13543841.0  10.178762 -42.0   804.0   
1     83891               83891    83891    108602.0   1.296400 -29.0  1536.0   
2    991986              991986   991986   8075787.0   8.155754 -43.0  1266.0   
3    946776              946776   946776  11235954.0  12.114915 -44.0  2315.0   
4    264816              264816   264816   1318662.0   5.034637 -56.0  1117.0   
5    625910              625910   625910   8072804.0  13.004564 -48.0  1525.0   
6    836445              836445   836445  10299232.0  12.564053 -82.0  2710.0   
7    257132              257132   257132   2593244.0  10.245765 -35.0  1506.0   
8    134683              134683   134683   2215632.0  17.214007 -30.0  1839.0   
9    105305              105305   105305   1059899.0  10.122910 -40.0  1979.0   
10   227888              227888   227888   3056324.0  13.803163 -36.0  2209.0   
11   289304              289304   289304   3028337.0  10.704734 -33.0  1844.0   
12   327007              327007   327007   2932330.0   9.272981 -36.0  2672.0   
13   297411              297411   297411   5212928.0  17.745473 -35.0  1769.0   
14   204845              204845   204845   2203398.0  10.940950 -46.0  1434.0   
15   135543              135543   135543   1944471.0  14.577009 -38.0  1022.0   
16   329149              329149   329149   2750223.0   8.544064 -62.0  1377.0   

             TAXI_OUT                      TAXI_IN                    \
   median         sum       mean median        sum       mean median   
0     0.0  16365531.0  12.301396   11.0  7108032.0   5.344940    4.0   
1    -3.0   1003874.0  11.984838   11.0   615110.0   7.344948    6.0   
2    -2.0  17062535.0  17.232186   15.0  7600296.0   7.676508    6.0   
3    -2.0  17211983.0  18.573274   16.0  8702242.0   9.392791    8.0   
4    -4.0   5068625.0  19.361638   17.0  2177245.0   8.326462    6.0   
5    -3.0  12404222.0  19.988820   17.0  4971548.0   8.014283    6.0   
6    -3.0  15758182.0  19.233785   17.0  6288526.0   7.687628    6.0   
7    -4.0   5246065.0  20.744044   17.0  1885489.0   7.457832    6.0   
8    -4.0   2728534.0  21.215072   18.0  1086869.0   8.454769    7.0   
9    -3.0   1361959.0  13.007832   11.0   712284.0   6.805695    6.0   
10   -3.0   3873211.0  17.504852   15.0  1628501.0   7.364417    6.0   
11   -3.0   5259935.0  18

In [39]:
OTP_carrier_2019.shape

(17, 39)

In [47]:
cols1 = ["".join(x) for i,x in enumerate(OTP_carrier_2019.columns.ravel()) if i <= 2]
cols2 = ["_".join(x) for i,x in enumerate(OTP_carrier_2019.columns.ravel()) if i > 2]
OTP_carrier_2019.columns = cols1 + cols2
OTP_carrier_2019

,OP_CARRIER_AIRLINE_ID,OP_UNIQUE_CARRIER,OP_CARRIER,ORIGIN_CITY_MARKET_ID_count,ORIGIN_count,DEST_CITY_MARKET_ID_count,DEST_count,DEP_DELAY_sum,DEP_DELAY_mean,DEP_DELAY_min,DEP_DELAY_max,DEP_DELAY_median,TAXI_OUT_sum,TAXI_OUT_mean,TAXI_OUT_median,TAXI_IN_sum,TAXI_IN_mean,TAXI_IN_median,ARR_DELAY_sum,ARR_DELAY_mean,ARR_DELAY_min,ARR_DELAY_max,ARR_DELAY_median,CANCELLED_sum,CANCELLED_<lambda_0>,DIVERTED_sum,DIVERTED_<lambda_0>,ACTUAL-CRS_ELAPSED_TIME_sum,ACTUAL-CRS_ELAPSED_TIME_mean,ACTUAL-CRS_ELAPSED_TIME_min,ACTUAL-CRS_ELAPSED_TIME_max,ACTUAL-CRS_ELAPSED_TIME_median,AIR_TIME_sum,AIR_TIME_mean,AIR_TIME_median,FLIGHTS_<lambda>,DISTANCE_sum,DISTANCE_mean,DISTANCE_median
0,19393,WN,WN,1363946,1363946,1363946,1363946,13543841.0,10.178762,-42.0,804.0,0.0,16365531.0,12.301396,11.0,7108032.0,5.344940,4.0,3421593.0,2.577351,-94.0,809.0,-6.0,33622.0,0.0,2762.0,0.0,-10054473.0,-7.573637,-91.0,194.0,-8.0,137045127.0,103.230679,90.0,1.0,1.011585e+09,741.660315,628.0
1,19690,HA,HA,83891,83891,83891,83891,108602.0,1.296400,-29.0,1536.0,-3.0,1003874.0,11.984838,11.0,615110.0,7.344948,6.0,53979.0,0.644995,-93.0,1507.0,-4.0,137.0,0.0,65.0,0.0,-52350.0,-0.625530,-89.0,105.0,-1.0,8289439.0,99.050520,28.0,1.0,6.297441e+07,750.669404,163.0
2,19790,DL,DL,991986,991986,991986,991986,8075787.0,8.155754,-43.0,1266.0,-2.0,17062535.0,17.232186,15.0,7600296.0,7.676508,6.0,1171704.0,1.185905,-88.0,1304.0,-8.0,1842.0,0.0,2119.0,0.0,-6823468.0,-6.906169,-96.0,227.0,-8.0,119925386.0,121.378898,99.0,1.0,8.892775e+08,896.461779,696.0
3,19805,AA,AA,946776,946776,946776,946776,11235954.0,12.114915,-44.0,2315.0,-2.0,17211983.0,18.573274,16.0,8702242.0,9.392791,8.0,6437726.0,6.965412,-87.0,2350.0,-5.0,20151.0,0.0,2383.0,0.0,-4675333.0,-5.058559,-81.0,279.0,-7.0,124008417.0,134.173103,121.0,1.0,9.383259e+08,991.074843,868.0
4,19930,AS,AS,264816,264816,264816,264816,1318662.0,5.034637,-56.0,1117.0,-4.0,5068625.0,19.361638,17.0,2177245.0,8.326462,6.0,352734.0,1.351394,-85.0,1087.0,-6.0,3077.0,0.0,724.0,0.0,-941620.0,-3.607532,-81.0,240.0,-4.0,45135699.0,172.923774,142.0,1.0,3.484294e+08,1315.741488,1009.0
5,19977,UA,UA,625910,625910,625910,625910,8072804.0,13.004564,-48.0,1525.0,-3.0,12404222.0,19.988820,17.0,4971548.0,8.014283,6.0,5128737.0,8.289444,-91.0,1523.0,-6.0,5384.0,0.0,1819.0,0.0,-2859008.0,-4.620940,-87.0,298.0,-7.0,96661244.0,156.231050,135.0,1.0,7.484520e+08,1195.782122,997.0
6,20304,OO,OO,836445,836445,836445,836445,10299232.0,12.564053,-82.0,2710.0,-3.0,15758182.0,19.233785,17.0,6288526.0,7.687628,6.0,5861765.0,7.184953,-84.0,2695.0,-8.0,17453.0,0.0,3153.0,0.0,-4317419.0,-5.291999,-221.0,203.0,-7.0,62133877.0,76.159484,68.0,1.0,4.106058e+08,490.893988,421.0
7,20363,9E,9E,257132,257132,257132,257132,2593244.0,10.245765,-35.0,1506.0,-4.0,5246065.0,20.744044,17.0,1885489.0,7.457832,6.0,773155.0,3.063844,-73.0,1511.0,-11.0,4257.0,0.0,527.0,0.0,-1786079.0,-7.077841,-67.0,175.0,-9.0,18326451.0,72.623722,68.0,1.0,1.192748e+08,463.866166,427.0
8,20366,EV,EV,134683,134683,134683,134683,2215632.0,17.214007,-30.0,1839.0,-4.0,2728534.0,21.215072,18.0,1086869.0,8.454769,7.0,1987662.0,15.506803,-61.0,1844.0,-4.0,6086.0,0.0,417.0,0.0,-205223.0,-1.601053,-127.0,1448.0,-4.0,9460320.0,73.804962,69.0,1.0,6.136961e+07,455.659630,427.0
9,20368,G4,G4,105305,105305,105305,105305,1059899.0,10.122910,-40.0,1979.0,-3.0,1361959.0,13.007832,11.0,712284.0,6.805695,6.0,836133.0,8.005639,-71.0,1966.0,-4.0,624.0,0.0,238.0,0.0,-217815.0,-2.085492,-66.0,188.0,-3.0,12150649.0,116.337610,115.0,1.0,9.132065e+07,867.201424,873.0


### Potentially relevant vizualizations VS A/L:

- Preferred city markets by A/L (Takeoff): ```'ORIGIN_CITY_MARKET_ID' : ['count']```
- Most used airports by A/L (Takeoff): ```'ORIGIN' : ['count']```
- Preferred city markets by A/L (Landing): ```'DEST_CITY_MARKET_ID' : ['count']```
- Most used airports by A/L (Landing): ```'DEST' : ['count']```
- Departure delay: ```'DEP_DELAY' : ['sum', 'mean', 'min', 'max', 'median']```
    - Sum: assess absolute operating cost increments due to departure delays
    - Mean: compare different average delays between A/Ls
    - Min: check how much in advance the A/Ls are able to prepare for a flight
    - Max: evaluate if extreme values of delays are moving excessively the mean value
    - Median: double-check if extreme values separate mean and median values too much
- Taxi-out operations: ```'TAXI_OUT' : ['sum', 'mean', 'median']```
    - Sum: assess absolute operating cost contribution of taxi-out operations
    - Mean: compare different average taxi-out times between A/Ls
    - Median: double-check if extreme values separate mean and median values too much
- Taxi-in operations: ```'TAXI_IN' : ['sum', 'mean', 'median']```
    - Sum: assess absolute operating cost contribution of taxi-in operations
    - Mean: compare different average taxi-in times between A/Ls
    - Median: double-check if extreme values separate mean and median values too much
- Arrival delay: ```'ARR_DELAY' : ['sum', 'mean', 'min', 'max', 'median']```
    - Sum: assess absolute operating cost increments due to arrival delays
    - Mean: compare different average delays between A/Ls
    - Min: check how much in advance the A/Ls are able to arrive at the destination airport
    - Max: evaluate if extreme values of delays are moving excessively the mean values
    - Median: double-check if extreme values separate mean and median values too much
- Cancelled flights: ```'CANCELLED' : ['sum', mode]```
    - Sum: assess absolute operating cost increments due to cancellations
    - Mode: ensure that '0' (i.e. not cancelled) is the most frequent value and investigate otherwise
- Diverted flights: ```'DIVERTED' : ['sum', mode]```
    - Sum: assess absolute operating cost increments due to diversions
    - Mode: ensure that '0' (i.e. not diverted) is the most frequent value and investigate otherwise
- Elapsed time difference (Actual - CRS): ```'ACTUAL-CRS_ELAPSED_TIME' : ['sum', 'mean', 'min', 'max', 'median']```
    - Sum: assess absolute operating cost increments due to overall delays
    - Mean: compare different average overall delays between A/Ls
    - Min: check how much in advance the A/Ls are able to arrive at the destination airport, considering also potentially advances at departure
    - Max: evaluate if extreme values of overall delays are moving excessively the mean values
    - Median: double-check if extreme values separate mean and median values too much
- Air time: ```'AIR_TIME' : ['sum', 'mean', 'median']```
    - Sum: assess absolute operating cost contribution of air time
    - Mean: compare different average air times between A/Ls
    - Median: double-check if extreme values separate mean and median values too much
- Number of flights *(legs)*: ```'FLIGHTS' : [mode]```
    - Mode: check how frequent is that flights consist of more than one leg
- Distance: ```'DISTANCE' : ['sum', 'mean', 'median']```
    - Sum: assess absolute operating cost contribution of distance
    - Mean: compare different average distances between A/Ls
    - Median: double-check if extreme values separate mean and median values too much

# 2. P12A_Fuel

In [ ]:
csv_path = os.path.join(root,
                        "Output_Data",
                        "US_DoT",
                        "P12A_fuel_output_2019.csv")
csv_path

In [ ]:
# Since 'pd.read_csv' works fine with zipped csv files, we can proceed directly:
cols = pd.read_csv(csv_path, nrows=1).columns # After normally importing it, an undesired extra blank column is loaded
df4 = pd.read_csv(csv_path,
                  encoding='latin1',
                  usecols=cols[:]) # This way, the extra column is disregarded for the loading process
df4

In [ ]:
df4.shape

When attempting to carry out the merge between the two datasets, the *validate* check showed the following error:
```
MergeError: Merge keys are not unique in right dataset; not a one-to-one merge
```
This means that, contrary to the initial premise of the column 'UNIQUE_CARRIER' having unique values, there must be some repeated values. Let's check.

In [ ]:
df4['UNIQUE_CARRIER'].value_counts()

Indeed, the 'EV' key appears twice in the Fuel dataset, let's further delve into it.

In [ ]:
df4.loc[df4['UNIQUE_CARRIER'] == 'EV', :]

Both records are identical, so let's simply drop the last one.

In [ ]:
df4.drop(index=49, axis=0, inplace=True)
df4.loc[df4['UNIQUE_CARRIER'] == 'EV', :]

# JOIN

The joining column will be one serving as a univocal carrier identifier. In this case:
```
- OTP :   'OP_UNIQUE_CARRIER'
- FUEL :  'UNIQUE_CARRIER'
```

Let's first check the values present in both tables:

In [ ]:
OTP_carriers = set(OTP_carrier_2019['OP_UNIQUE_CARRIER'])
Fuel_carriers = set(df4['UNIQUE_CARRIER'])
print(len(OTP_carriers), OTP_carriers)
print(len(Fuel_carriers), Fuel_carriers)
print(len(OTP_carriers - Fuel_carriers), OTP_carriers - Fuel_carriers)
print(len(Fuel_carriers - OTP_carriers), Fuel_carriers - OTP_carriers)

The analysis will therefore focus on the 17 airlines that appear in both datasets.

In [ ]:
OTP_Fuel = OTP_carrier_2019.merge(right=df4,
                                  how='inner',
                                  left_on='OP_UNIQUE_CARRIER',
                                  right_on='UNIQUE_CARRIER',
                                  validate="1:1")
OTP_Fuel